In [422]:
import pymongo

In [423]:
#The process to connect to the MongoDB Atlas is concealed because of the privacy
#client = pymongo.MongoClient(<concealed>) 
db = client["myFirstDatabase"]

## Q1

In [425]:
db.orders.insert_many( [
{ "_id": 0, "productName": "Steel beam", "status": "new", "quantity": 10 },
{ "_id": 1, "productName": "Steel beam", "status": "urgent", "quantity": 20 },
{ "_id": 2, "productName": "Steel beam", "status": "urgent", "quantity": 30 },
{ "_id": 3, "productName": "Iron rod", "status": "new", "quantity": 15 },
{ "_id": 4, "productName": "Iron rod", "status": "urgent", "quantity": 50 },
{ "_id": 5, "productName": "Iron rod", "status": "urgent", "quantity": 10 }
] )


### a

In [426]:
pipeline =[{"$group":{"_id":'$productName',"sumQuantity":{"$sum":'$quantity'}}},
          {"$sort":{'sumQuantity':-1}}]

In [427]:
for i in db['orders'].aggregate(pipeline):
 print(i)

{'_id': 'Iron rod', 'sumQuantity': 75}
{'_id': 'Steel beam', 'sumQuantity': 60}


There are two stages - group and sort.

### b

In [428]:
pipeline =[{"$match":{"status":{"$eq":"urgent"}}},
           {"$group":{"_id":'$productName',"sumQuantity":{"$sum":'$quantity'}}},
          {"$sort":{'sumQuantity':-1}}]

In [429]:
for i in db['orders'].aggregate(pipeline):
 print(i)

{'_id': 'Iron rod', 'sumQuantity': 60}
{'_id': 'Steel beam', 'sumQuantity': 50}


There are three stages - match, group and sort.

### c

In [430]:
pipeline =[{"$group":{"_id":['$productName',"$status"],"sumQuantity":{"$sum":'$quantity'}}},
          {"$sort":{"_id": 1}}]

In [431]:
for i in db['orders'].aggregate(pipeline):
 print(i)

{'_id': ['Iron rod', 'new'], 'sumQuantity': 15}
{'_id': ['Iron rod', 'urgent'], 'sumQuantity': 60}
{'_id': ['Steel beam', 'new'], 'sumQuantity': 10}
{'_id': ['Steel beam', 'urgent'], 'sumQuantity': 50}


There are two stages - group and sort.

### d

In [432]:
pipeline =[
    {"$group":{"_id":['$productName',"$status"],"name":{"$first":"$productName"},"status":{"$first":"$status"},"sumQuantity":{"$sum":'$quantity'}}},
    {"$sort":{"name":1,"status":-1}},
    {"$unset":["name","status"]},
    {"$limit":3}
           ]

In [433]:
for i in db['orders'].aggregate(pipeline):
    print(i)

{'_id': ['Iron rod', 'urgent'], 'sumQuantity': 60}
{'_id': ['Iron rod', 'new'], 'sumQuantity': 15}
{'_id': ['Steel beam', 'urgent'], 'sumQuantity': 50}


There are four stages - group, sort, unset and limit.

## Q2

In [434]:
db = client["sample_training"]

### a

In [435]:
pipeline=[
{"$group":{"_id":"$class_id"}},
{"$group":{"_id":"total_class","count":{"$sum":1}}}    
]

In [436]:
for i in db['grades'].aggregate(pipeline):
    print(i)

{'_id': 'total_class', 'count': 501}


There are 501 classes.

### b

In [437]:
pipeline=[
{"$group":{"_id":"$student_id"}},
{"$group":{"_id":"total_student","count":{"$sum":1}}}    
]

In [438]:
for i in db['grades'].aggregate(pipeline):
    print(i)

{'_id': 'total_student', 'count': 10000}


There are 10000 students.

### c

In [439]:
pipeline=[
{"$match":{"class_id":{"$eq":212}}},
{"$unwind":"$scores"},
{"$match":{"scores.type":{"$eq":'exam'}}},
{"$group":{"_id":None,"avg_exam_score":{"$avg":"$scores.score"}}} 
]

In [440]:
for i in db['grades'].aggregate(pipeline):
    print(i)

{'_id': None, 'avg_exam_score': 49.32031796981391}


The average exam scores of class 212 is 49.32.

### d

In [441]:
pipeline=[
{"$match":{"class_id":{"$eq":212}}},
{"$unwind":"$scores"},
{"$match":{"scores.type":{"$eq":'exam'}}},
{"$group":{"_id":None,"std_exam_score":{"$stdDevPop":"$scores.score"}}} 
]

In [442]:
for i in db['grades'].aggregate(pipeline):
    print(i)

{'_id': None, 'std_exam_score': 29.280553806350348}


The standard deviation of exam scores of class 212 is 29.28.